In [ ]:
# 소방청 민원처리 AI Agent 교육 (강사 버전)
# 실습 과정 - Colab 전용

"""
교육 목표: 소방 민원 자동 처리 시스템 구축
- RAG 기반 법령/가이드 검색
- AI 보고서 자동 생성
- Gradio UI 구현

Lab 구성:
1. 민원 자동 분류 (API 실습 포함)
2. RAG 기초 - 소방기본법 (10p)
3. 고급 검색 - 화재예방 (44p)
4. 보고서 자동 생성
5. Gradio UI + 최종 통합
"""

# ============================================================
# 🔧 환경 설정
# ============================================================

import sys,os
if 'google.colab' in sys.modules:
    !pip install openai chromadb pypdf2 gradio python-dotenv pdfplumber -q
    #!pip install openai==1.12.0 chromadb==0.4.22 pypdf2==3.0.1 gradio==4.19.0 python-dotenv==1.0.0 -q

import os
import getpass
from openai import OpenAI
import numpy as np
import pdfplumber
import chromadb
from chromadb.config import Settings
import gradio as gr
from typing import List, Dict
import time

# API 키 설정
print(" OpenAI API 키를 입력하세요")
my_api_key = "".strip()

#api_key = getpass.getpass(my_api_key)
client = OpenAI(api_key=my_api_key)

print(" 환경 설정 완료!")

In [ ]:

# ============================================================
# Lab 1: 민원 자동 분류 + API 실습 (40분)
# ============================================================

print("\n" + "="*60)
print("[Lab 1] 민원 자동 분류 + OpenAI API 실습")
print("="*60)

# 실습용 민원 샘플
sample_complaints = [
    "아파트 복도에 소화기가 없는데 괜찮은가요?",
    "식당을 운영하는데 소방시설 점검은 언제 받아야 하나요?",
    "옆집에서 불이 났는데 어디로 신고해야 하나요?",
    "소방차 전용 주차구역에 차를 세웠다가 과태료 부과 통보를 받았습니다.",
]

# ------------------------------------
# 1-1. 기본 API 호출
# ------------------------------------

def classify_complaint_basic(text: str) -> str:
    """
    민원을 카테고리별로 분류하는 기본 함수
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """당신은 소방청 민원 분류 전문가입니다.
민원을 다음 카테고리로 분류하세요:
1. 화재신고/대응
2. 소방시설 점검/설치
3. 법규 위반/과태료
4. 일반 문의

카테고리와 간단한 이유를 반환하세요."""
            },
            {"role": "user", "content": text}
        ],
        temperature=0.3,
        max_tokens=150
    )
    return response.choices[0].message.content

# 테스트
print("\n[기본 분류 테스트]")
test_complaint = sample_complaints[0]
print(f"민원: {test_complaint}")
print(f"분류 결과:\n{classify_complaint_basic(test_complaint)}\n")


In [ ]:

# ------------------------------------
# 1-2. 온도(Temperature) 비교 실습
# ------------------------------------

print("\n[실습 1-2] Temperature 비교")
print("-" * 50)

def compare_temperature(text: str):
    """
    동일 질문에 대해 온도를 다르게 설정하여 응답 비교
    """
    temperatures = [0.0, 0.5, 1.0]

    for temp in temperatures:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "소방 민원을 분류하세요."},
                {"role": "user", "content": text}
            ],
            temperature=temp,
            max_tokens=100
        )
        print(f"\nTemperature = {temp}")
        print(f"응답: {response.choices[0].message.content}")
        print(f"토큰 사용: {response.usage.total_tokens}")

# 실행
compare_temperature("소화기 사용법을 알려주세요")


In [ ]:

# ------------------------------------
# 1-3. 토큰 제한 실습
# ------------------------------------

print("\n\n[실습 1-3] Max Tokens 비교")
print("-" * 50)

def compare_max_tokens(text: str):
    """
    토큰 제한에 따른 응답 차이 확인
    """
    token_limits = [50, 150, 500]

    for max_tok in token_limits:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "소방 민원을 상세히 분류하고 처리 방안을 제시하세요."},
                {"role": "user", "content": text}
            ],
            temperature=0.5,
            max_tokens=max_tok
        )
        print(f"\nMax Tokens = {max_tok}")
        print(f"응답 길이: {len(response.choices[0].message.content)} 문자")
        print(f"응답: {response.choices[0].message.content[:200]}...")

# 실행
compare_max_tokens("아파트 복도 소화기 설치 기준과 위반 시 처리 방법을 알려주세요")


In [ ]:

# ------------------------------------
# 1-4. 스트리밍 vs 일반 응답
# ------------------------------------

print("\n\n[실습 1-4] 스트리밍 vs 일반 응답 속도 비교")
print("-" * 50)

# 일반 응답
start = time.time()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "소방시설 점검 주기를 설명하세요"}],
    stream=False
)
normal_time = time.time() - start
print(f"[일반 응답] 소요 시간: {normal_time:.2f}초")
print(f"응답: {response.choices[0].message.content}\n")

# 스트리밍 응답
print("[스트리밍 응답]")
start = time.time()
stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "소방시설 점검 주기를 설명하세요"}],
    stream=True
)
for chunk in stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
stream_time = time.time() - start
print(f"\n\n[스트리밍 완료] 소요 시간: {stream_time:.2f}초")
print(f"참고: 스트리밍이 더 빠르게 느껴집니다!")


In [ ]:

# ------------------------------------
# 1-5. 최종 분류 함수 (Lab 2-5에서 재사용)
# ------------------------------------

def classify_complaint(text: str) -> Dict[str, str]:
    """
    민원 분류 - 구조화된 결과 반환
    Lab 2-5에서 재사용할 핵심 함수
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """소방 민원을 분류하고 다음 형식으로 답변하세요:

카테고리: [화재신고/소방시설/법규위반/일반문의]
긴급도: [높음/보통/낮음]
키워드: [핵심 키워드 3개]
요약: [한 줄 요약]"""
            },
            {"role": "user", "content": text}
        ],
        temperature=0.3,
        max_tokens=200
    )

    result_text = response.choices[0].message.content

    # 파싱
    lines = result_text.strip().split('\n')
    result = {
        "원문": text,
        "분류_결과": result_text,
        "전체_응답": result_text
    }

    return result

# 전체 샘플 분류 테스트
print("\n\n[Lab 1 최종] 모든 샘플 분류")
print("="*60)

for i, complaint in enumerate(sample_complaints, 1):
    print(f"\n{i}. 민원: {complaint}")
    result = classify_complaint(complaint)
    print(f"분류:\n{result['분류_결과']}")
    print("-"*60)

print("\n[완료] Lab 1 완료! classify_complaint() 함수는 Lab 2-5에서 계속 사용됩니다.")


In [ ]:

# ============================================================
# Lab 2: RAG 기초 - 소방기본법 (40분)
# ============================================================

print("\n" + "="*60)
print("[Lab 2] RAG 기초 - 소방기본법 검색")
print("="*60)
# 한글 처리 설정
os.environ["PYTHONIOENCODING"] = "utf-8"

# ------------------------------------
# 2-1. PDF 읽기 및 텍스트 추출
# ------------------------------------
def extract_text_from_pdf(pdf_path, max_pages=None):
    pages_data = []
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages) if max_pages is None else min(max_pages, len(pdf.pages))
        for i in range(total_pages):
            text = pdf.pages[i].extract_text()
            pages_data.append({
                "page_num": i + 1,
                "text": text,
                "char_count": len(text)
            })
    return pages_data

# 소방기본법 읽기
print("\n[진행중] 소방기본법_28조.pdf 읽기...")
#fire_law_pages = extract_text_from_pdf("소방기본법_28조.pdf", max_pages=10)
fire_law_pages = extract_text_from_pdf("소방기본법_28조.pdf", max_pages=None)
max_pages={len(fire_law_pages)}
print(f"[완료] 총 {len(fire_law_pages)} 페이지 추출")
print(f"\n[샘플] 1페이지 샘플 (처음 300자):")
print(fire_law_pages[0]['text'][:300])

# ------------------------------------
# 2-2. 텍스트 청킹 (Chunking)
# ------------------------------------

def chunk_text(pages_data: List[Dict], chunk_size: int = 500, overlap: int = 50) -> List[Dict]:
    """
    텍스트를 작은 청크로 분할
    """
    chunks = []
    chunk_id = 0

    for page_data in pages_data:
        text = page_data['text']
        page_num = page_data['page_num']

        # 간단한 청킹 (문자 기반)
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk_text = text[start:end]

            if chunk_text.strip():  # 빈 청크 제외
                chunks.append({
                    "id": f"law_chunk_{chunk_id}",
                    "text": chunk_text,
                    "page": page_num,
                    "start_pos": start
                })
                chunk_id += 1

            start = end - overlap

    return chunks

# 청킹 실행
print("\n[진행중] 텍스트 청킹 중...")
law_chunks = chunk_text(fire_law_pages, chunk_size=500, overlap=50)

print(f"[완료] 총 {len(law_chunks)}개 청크 생성")
print(f"\n[샘플] 첫 번째 청크:")
print(f"ID: {law_chunks[0]['id']}")
print(f"페이지: {law_chunks[0]['page']}")
print(f"내용: {law_chunks[0]['text'][:200]}...")


In [ ]:

# ------------------------------------
# 2-3. 임베딩 생성
# ------------------------------------

def get_embedding(text: str) -> List[float]:
    """
    텍스트를 벡터로 변환
    """
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding

# 샘플 임베딩 테스트
print("\n[진행중] 임베딩 테스트...")
sample_text = "소방시설 설치 기준"
sample_embedding = get_embedding(sample_text)
print(f"[완료] 임베딩 차원: {len(sample_embedding)}")
print(f"샘플 벡터 (처음 5개): {sample_embedding[:5]}")


In [ ]:

# ------------------------------------
# 2-4. ChromaDB에 저장
# ------------------------------------

print("\n[진행중] ChromaDB 생성 중...")

# ChromaDB 클라이언트 생성
chroma_client = chromadb.Client(Settings(
    persist_directory="./chroma_db",
    anonymized_telemetry=False
))

# 컬렉션 생성 (기존 것 삭제 후)
try:
    chroma_client.delete_collection("fire_law")
except:
    pass

fire_law_collection = chroma_client.create_collection(
    name="fire_law",
    metadata={"description": "소방기본법"}
)

# 청크 임베딩 및 저장
print("[진행중] 청크 임베딩 및 저장 중...")
for i, chunk in enumerate(law_chunks):
    if i % 10 == 0:
        print(f"  진행: {i}/{len(law_chunks)}")

    embedding = get_embedding(chunk['text'])

    fire_law_collection.add(
        ids=[chunk['id']],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{
            "page": chunk['page'],
            "source": "소방기본법"
        }]
    )

print(f"[완료] {len(law_chunks)}개 청크 저장 완료!")


In [ ]:

# ------------------------------------
# 2-5. RAG 검색 함수 (Lab 4에서 재사용)
# ------------------------------------

def search_fire_law(query: str, n_results: int = 3) -> List[Dict]:
    """
    소방기본법 검색 함수
    Lab 4에서 보고서 생성 시 재사용
    """
    query_embedding = get_embedding(query)

    results = fire_law_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    search_results = []
    for i in range(len(results['documents'][0])):
        search_results.append({
            "text": results['documents'][0][i],
            "page": results['metadatas'][0][i]['page'],
            "source": results['metadatas'][0][i]['source']
        })

    return search_results

# ------------------------------------
# 2-6. RAG 기반 답변 생성
# ------------------------------------

def answer_with_rag(question: str) -> str:
    """
    질문 -> 검색 -> 답변 생성
    """
    # 1. 관련 문서 검색
    search_results = search_fire_law(question, n_results=3)

    # 2. 컨텍스트 구성
    context = "\n\n".join([
        f"[출처: {r['source']}, {r['page']}페이지]\n{r['text']}"
        for r in search_results
    ])

    # 3. LLM에게 답변 요청
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """당신은 소방법령 전문가입니다.
제공된 문서를 기반으로 정확하게 답변하세요.
문서에 없는 내용은 "제공된 자료에서 찾을 수 없습니다"라고 하세요."""
            },
            {
                "role": "user",
                "content": f"""다음 문서를 참고하여 질문에 답하세요.

[참고 문서]
{context}

[질문]
{question}"""
            }
        ],
        temperature=0.3,
        max_tokens=500
    )

    return response.choices[0].message.content

# 테스트
print("\n\n[Lab 2 최종] RAG 검색 테스트")
print("="*60)

test_questions = [
    "소방시설 설치 대상은 어떻게 되나요?",
    "소방시설 점검은 언제 해야 하나요?",
    "위반 시 과태료는 얼마인가요?"
]

for q in test_questions:
    print(f"\n[질문] {q}")
    print("-"*50)

    # 검색 결과 확인
    results = search_fire_law(q, n_results=2)
    print("[검색된 문서]")
    for i, r in enumerate(results, 1):
        print(f"  {i}. {r['source']} {r['page']}페이지")
        print(f"     {r['text'][:150]}...")

    # 답변 생성
    print("\n[답변]")
    answer = answer_with_rag(q)
    print(answer)
    print("="*60)

print("\n[완료] Lab 2 완료! search_fire_law() 함수는 Lab 4에서 재사용됩니다.")


In [ ]:

# ============================================================
# Lab 3: 고급 검색 - 화재예방 (40분)
# ============================================================

print("\n" + "="*60)
print("[Lab 3] 고급 검색 - 화재예방 가이드")
print("="*60)

# ------------------------------------
# 3-1. 화재예방 PDF 처리
# ------------------------------------

print("\n[진행중] 화재예방.pdf 읽기...")
prevention_pages = extract_text_from_pdf("화재예방.pdf")

print(f"[완료] 총 {len(prevention_pages)} 페이지 추출")

# 청킹
print("\n[진행중] 텍스트 청킹 중...")
prevention_chunks = chunk_text(prevention_pages, chunk_size=500, overlap=50)
print(f"[완료] 총 {len(prevention_chunks)}개 청크 생성")


In [ ]:

# ------------------------------------
# 3-2. 두 번째 컬렉션 생성
# ------------------------------------

print("\n[진행중] 화재예방 ChromaDB 생성 중...")

try:
    chroma_client.delete_collection("fire_prevention")
except:
    pass

prevention_collection = chroma_client.create_collection(
    name="fire_prevention",
    metadata={"description": "화재예방 가이드"}
)

# 저장
print("[진행중] 청크 임베딩 및 저장 중...")
for i, chunk in enumerate(prevention_chunks):
    if i % 20 == 0:
        print(f"  진행: {i}/{len(prevention_chunks)}")

    embedding = get_embedding(chunk['text'])

    prevention_collection.add(
        ids=[chunk['id']],
        embeddings=[embedding],
        documents=[chunk['text']],
        metadatas=[{
            "page": chunk['page'],
            "source": "화재예방가이드"
        }]
    )

print(f"[완료] {len(prevention_chunks)}개 청크 저장 완료!")


In [ ]:

# ------------------------------------
# 3-3. 화재예방 검색 함수 (Lab 4에서 재사용)
# ------------------------------------

def search_fire_prevention(query: str, n_results: int = 3) -> List[Dict]:
    """
    화재예방 가이드 검색
    Lab 4에서 보고서 생성 시 재사용
    """
    query_embedding = get_embedding(query)

    results = prevention_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    search_results = []
    for i in range(len(results['documents'][0])):
        search_results.append({
            "text": results['documents'][0][i],
            "page": results['metadatas'][0][i]['page'],
            "source": results['metadatas'][0][i]['source']
        })

    return search_results

# ------------------------------------
# 3-4. 통합 검색 (법령 + 가이드)
# ------------------------------------

def search_both_sources(query: str, n_results_per_source: int = 2) -> Dict:
    """
    소방기본법 + 화재예방 가이드 동시 검색
    Lab 4에서 보고서 작성 시 사용
    """
    law_results = search_fire_law(query, n_results_per_source)
    prevention_results = search_fire_prevention(query, n_results_per_source)

    return {
        "법령": law_results,
        "예방가이드": prevention_results,
        "total_count": len(law_results) + len(prevention_results)
    }

# 테스트
print("\n\n[Lab 3 최종] 통합 검색 테스트")
print("="*60)

test_queries = [
    "소화기 관리 방법",
    "화재 발생 시 대응 절차",
]

for query in test_queries:
    print(f"\n[검색어] {query}")
    print("-"*50)

    combined_results = search_both_sources(query, n_results_per_source=2)

    print("\n[소방기본법]")
    for i, r in enumerate(combined_results['법령'], 1):
        print(f"  {i}. {r['page']}페이지")
        print(f"     {r['text'][:120]}...")

    print("\n[화재예방가이드]")
    for i, r in enumerate(combined_results['예방가이드'], 1):
        print(f"  {i}. {r['page']}페이지")
        print(f"     {r['text'][:120]}...")

    print("="*60)

print("\n[완료] Lab 3 완료! search_both_sources() 함수는 Lab 4에서 재사용됩니다.")


In [ ]:

# ============================================================
# Lab 4: 보고서 자동 생성 (40분)
# ============================================================

print("\n" + "="*60)
print("[Lab 4] 민원 처리 보고서 자동 생성")
print("="*60)

# ------------------------------------
# 4-1. 보고서 생성 함수 (Lab 1-3 통합)
# ------------------------------------

def generate_complaint_report(complaint_text: str) -> Dict:
    """
    민원 -> 분류 -> 검색 -> 보고서 생성
    Lab 1, 2, 3의 모든 함수를 활용
    """
    print("\n[진행중] 보고서 생성 중...")

    # Step 1: 민원 분류 (Lab 1)
    print("  [1단계] 민원 분류 중...")
    classification = classify_complaint(complaint_text)

    # Step 2: 관련 자료 검색 (Lab 2, 3)
    print("  [2단계] 관련 법령 및 가이드 검색 중...")
    search_results = search_both_sources(complaint_text, n_results_per_source=2)

    # Step 3: 보고서 본문 생성
    print("  [3단계] 보고서 작성 중...")

    # 컨텍스트 구성
    law_context = "\n".join([
        f"- {r['source']} {r['page']}p: {r['text'][:200]}"
        for r in search_results['법령']
    ])

    prevention_context = "\n".join([
        f"- {r['source']} {r['page']}p: {r['text'][:200]}"
        for r in search_results['예방가이드']
    ])

    # LLM으로 보고서 생성
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": """당신은 소방청 민원 처리 담당자입니다.
다음 형식으로 공식 보고서를 작성하세요:

【민원 처리 보고서】

1. 민원 내용
   [민원 요약]

2. 분류 결과
   [카테고리, 긴급도 등]

3. 관련 법령 및 근거
   [소방기본법 내용]

4. 처리 방안
   [구체적인 대응 방법]

5. 예방 가이드
   [화재예방 참고사항]

6. 담당 부서 및 처리 기한
   [추천 부서 및 기한]
"""
            },
            {
                "role": "user",
                "content": f"""다음 민원에 대한 처리 보고서를 작성하세요.

[민원 내용]
{complaint_text}

[분류 정보]
{classification['분류_결과']}

[참고 - 소방기본법]
{law_context}

[참고 - 화재예방 가이드]
{prevention_context}
"""
            }
        ],
        temperature=0.4,
        max_tokens=1000
    )

    report = response.choices[0].message.content

    print("  [완료] 보고서 생성 완료!")

    return {
        "민원원문": complaint_text,
        "분류": classification,
        "법령검색": search_results['법령'],
        "예방검색": search_results['예방가이드'],
        "보고서": report
    }

# ------------------------------------
# 4-2. 보고서 생성 테스트
# ------------------------------------

print("\n\n[Lab 4 최종] 보고서 생성 테스트")
print("="*60)

test_complaints_for_report = [
    "상가 건물 복도에 소화기가 없고, 비상구도 짐으로 막혀있습니다. 안전 점검을 요청합니다.",
    "음식점을 새로 열려고 하는데 소방시설을 어떻게 설치해야 하나요?"
]

for i, complaint in enumerate(test_complaints_for_report, 1):
    print(f"\n\n{'='*60}")
    print(f"[테스트 {i}]")
    print(f"{'='*60}")
    print(f"민원: {complaint}\n")

    result = generate_complaint_report(complaint)

    print("\n" + "="*60)
    print("[생성된 보고서]")
    print("="*60)
    print(result['보고서'])
    print("\n" + "="*60)

print("\n[완료] Lab 4 완료! generate_complaint_report() 함수는 Lab 5에서 UI에 연결됩니다.")


In [ ]:

# ============================================================
# Lab 5: Gradio UI + 최종 통합 (40분)
# ============================================================

print("\n" + "="*60)
print("[Lab 5] Gradio UI 구축 및 최종 완성")
print("="*60)

# ------------------------------------
# 5-1. Gradio UI 함수
# ------------------------------------

def process_complaint_ui(complaint_text: str) -> str:
    """
    Gradio UI용 래퍼 함수
    """
    if not complaint_text.strip():
        return "[경고] 민원 내용을 입력해주세요."

    try:
        result = generate_complaint_report(complaint_text)
        return result['보고서']
    except Exception as e:
        return f"[오류 발생] {str(e)}"

# ------------------------------------
# 5-2. Gradio 인터페이스 구축
# ------------------------------------

# 예시 민원들
examples = [
    ["아파트 복도에 소화기가 없는데 괜찮은가요?"],
    ["식당을 운영하는데 소방시설 점검은 언제 받아야 하나요?"],
    ["소방차 전용구역에 주차했다가 과태료를 받았습니다."],
]

# Gradio 인터페이스
interface = gr.Interface(
    fn=process_complaint_ui,
    inputs=gr.Textbox(
        label="민원 내용",
        placeholder="처리할 민원 내용을 입력하세요...",
        lines=5
    ),
    outputs=gr.Textbox(
        label="처리 보고서",
        lines=20
    ),
    title="소방청 민원처리 AI Agent",
    description="""
    **민원 내용을 입력하면 자동으로 다음을 수행합니다:**
    1. 민원 자동 분류
    2. 관련 법령 검색 (소방기본법)
    3. 예방 가이드 검색 (화재예방)
    4. 공식 처리 보고서 생성

    아래 예시를 클릭하거나 직접 입력해보세요!
    """,
    examples=examples,
    theme=gr.themes.Soft()
)

# ------------------------------------
# 5-3. UI 실행
# ------------------------------------

print("\n[실행중] Gradio UI 실행 중...")
print("\n" + "="*60)
print("[완료] 모든 Lab 완료!")
print("="*60)
print("""
학습 내용 정리:
  Lab 1: OpenAI API 사용법 + 민원 분류
  Lab 2: RAG 기초 + 소방기본법 검색
  Lab 3: 다중 문서 검색 + 화재예방 가이드
  Lab 4: 통합 보고서 자동 생성
  Lab 5: Gradio UI 구축

이제 실전 민원을 처리해보세요!
""")

# Gradio 실행
interface.launch(share=True)